In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import puz
import os

from pprint import pprint
from collections import defaultdict

In [2]:
all_txt_files = glob.glob('./Second pass model outputs/*.txt')
for txt_files in all_txt_files:
    if '_test.txt' in txt_files:
        print(txt_files)

./Second pass model outputs\atlantic_crossword_test.txt
./Second pass model outputs\newsday_crossword_test.txt
./Second pass model outputs\nyt_crossword_test.txt
./Second pass model outputs\thelatimes_crossword_test.txt
./Second pass model outputs\thenewyorker_crossword_test.txt


In [3]:
TEST_PUB_PATHS = {
    'Atlantic': "./Second pass model outputs/atlantic_crossword_test.txt",
    'News Day' : "./Second pass model outputs/newsday_crossword_test.txt",
    "The LA Times" : "./Second pass model outputs/thelatimes_crossword_test.txt",
    "The New Yorker" : "./Second pass model outputs/thenewyorker_crossword_test.txt"
#     "New York Times" : "./Second pass model outputs/nyt_crossword_test.txt"
}

In [4]:
def extract_float(input_string):
    pattern = r"\d+\.\d+"
    matches = re.findall(pattern, input_string)
    float_numbers = [float(match) for match in matches]
    return float_numbers

In [5]:
def extract_data(lines):
    date_pattern = r"\b\d{2}/\d{2}/\d{4}\b"
    data_dict = {}
    new_date_math = False
    current_date = ''
    error_dates = []

    for line in lines:
        if 'error' in line:
            match = re.search(date_pattern, line)
            error_dates.append(match.group())

    for line in lines:
        match = re.match(date_pattern, line) # to match the date pattern

        if match:
            current_date = line.strip()
            data_dict[current_date] = {}
            data_dict[current_date]['Letter II'] = []
            data_dict[current_date]['Word II'] = []


        if 'Before' in line:
            [lett_accu, word_accu] = extract_float(line)
            data_dict[current_date]['Before Letter Accuracy'] = lett_accu
            data_dict[current_date]['Before Word Accuracy'] = word_accu

        if 'iteration:' in line:
            lett_accu, word_accu = extract_float(line)
            data_dict[current_date]['Letter II'].append(lett_accu)
            data_dict[current_date]['Word II'].append(word_accu)
    return data_dict

In [6]:
def get_df_data(data_dict, threshold = 85.0):
    output_data = []
    for date, inf_data in data_dict.items():

        if len(inf_data.keys()) < 4:
            continue

        # only first pass model output exists
        if len(inf_data['Letter II']) == 0:
            f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
            f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

            s_pass_l_accu = f_pass_l_accu
            s_pass_w_accu = f_pass_w_accu
            output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
        else:
            f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
            f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

            s_pass_l_list = inf_data['Letter II']
            s_pass_w_list = inf_data['Word II']
            max_accu_index = s_pass_l_list.index(max(s_pass_l_list))

            s_pass_l_accu = round(s_pass_l_list[max_accu_index], 2)
            s_pass_w_accu = round(s_pass_w_list[max_accu_index], 2)

            if s_pass_l_accu < f_pass_l_accu:
                s_pass_l_accu = f_pass_l_accu
                s_pass_w_accu = f_pass_w_accu

            output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
            
    df = pd.DataFrame(output_data, columns = ['Date', 'First Pass Letter Accuracy', 'First Pass Word Accuracy', 'Second Pass Letter Accuracy', 'Second Pass Word Accuracy'])

    # taking a heuristic that, word accuracies lower than 85 are invalid due to crossword invalidity
    df = df[df['First Pass Word Accuracy'] > threshold]
    
    # first find the number of cells which has 100% accuracies`
    pp_count_by_f_pass = len(df[df['First Pass Letter Accuracy'] == 100.0])
    pp_count_by_s_pass = len(df[df['Second Pass Letter Accuracy'] == 100.0])
    total_test_size = len(df)
    
    avg_pp_accu_f_pass = round((pp_count_by_f_pass / total_test_size) * 100, 2)
    avg_pp_accu_s_pass = round((pp_count_by_s_pass / total_test_size) * 100, 2)

    f_pass_lett = round(np.mean(df['First Pass Letter Accuracy']), 2)
    f_pass_word = round(np.mean(df['First Pass Word Accuracy']), 2)

    s_pass_lett = round(np.mean(df['Second Pass Letter Accuracy']), 2)
    s_pass_word = round(np.mean(df['Second Pass Word Accuracy']), 2)
    print('Total Number of puzzles: ', total_test_size)
    print("Perfect Puzzle Accuracy (First Pass Model): ", avg_pp_accu_f_pass)
    print("Perfect Puzzle Accuracy (Second Pass Model): ", avg_pp_accu_s_pass)

    print("\nLetter Accuracy (First Pass Model): ", f_pass_lett)
    print("Word Accuracy (First Pass Model): ", f_pass_word)

    print("\nLetter Accuracy (Second Pass Model): ", s_pass_lett)
    print("Word Accuracy (Second Pass Model): ", s_pass_word)
    
    return [f_pass_lett, f_pass_word, avg_pp_accu_f_pass, s_pass_lett, s_pass_word, avg_pp_accu_s_pass]

In [7]:
thresholds = [85.0, 85.0, 0, 90.0, 80.0]
accumulative_data = []
for i, (publication, path) in enumerate(TEST_PUB_PATHS.items()):
    if i == 0:
        print('-'*100)
    print(publication, end = '\n\n')
    txt_path = TEST_PUB_PATHS[publication]
    all_lines = open(txt_path, 'r').readlines()

    data_dict = extract_data(all_lines)
    main_data = get_df_data(data_dict, thresholds[i])
    
    accumulative_data.append((publication, *main_data))
    print("-"*100)

----------------------------------------------------------------------------------------------------
Atlantic

Total Number of puzzles:  89
Perfect Puzzle Accuracy (First Pass Model):  76.4
Perfect Puzzle Accuracy (Second Pass Model):  91.01

Letter Accuracy (First Pass Model):  99.49
Word Accuracy (First Pass Model):  97.8

Letter Accuracy (Second Pass Model):  99.86
Word Accuracy (Second Pass Model):  99.34
----------------------------------------------------------------------------------------------------
News Day

Total Number of puzzles:  100
Perfect Puzzle Accuracy (First Pass Model):  75.0
Perfect Puzzle Accuracy (Second Pass Model):  90.0

Letter Accuracy (First Pass Model):  99.73
Word Accuracy (First Pass Model):  98.91

Letter Accuracy (Second Pass Model):  99.93
Word Accuracy (Second Pass Model):  99.66
----------------------------------------------------------------------------------------------------
The LA Times

Total Number of puzzles:  140
Perfect Puzzle Accuracy (Fir

In [8]:
last_df = pd.DataFrame(accumulative_data, columns = ['Source', 'First Pass Letter Accuracy', 'First Pass Word Accuracy', 'First Pass Perfect Puzzle Accuracy', 'Second Pass Letter Accuracy', 'Second Pass Word Accuracy', 'Second Pass Perfect Puzzle Accuracy'])
# last_df.to_csv("./Second pass model outputs/all_publication_inference_test.csv")

### Captures the Letter, Word and Perfect Puzzle Accuracies based on the Corresponding Grid Size

In [9]:
def puz_to_json(fname):
    """ Converts a puzzle in .puz format to .json format
    """
    p = puz.read(fname)
    numbering = p.clue_numbering()

    grid = [[None for _ in range(p.width)] for _ in range(p.height)]
    for row_idx in range(p.height):
        cell = row_idx * p.width
        row_solution = p.solution[cell:cell + p.width]
        for col_index, item in enumerate(row_solution):
            if p.solution[cell + col_index:cell + col_index + 1] == '.':
                grid[row_idx][col_index] = 'BLACK'
            else:
                grid[row_idx][col_index] = ["", row_solution[col_index: col_index + 1]]

    across_clues = {}
    for clue in numbering.across:
        answer = ''.join(p.solution[clue['cell'] + i] for i in range(clue['len']))
        across_clues[str(clue['num'])] = [clue['clue'] + ' ', ' ' + answer]
        grid[int(clue['cell'] / p.width)][clue['cell'] % p.width][0] = str(clue['num'])

    down_clues = {}
    for clue in numbering.down:
        answer = ''.join(p.solution[clue['cell'] + i * numbering.width] for i in range(clue['len']))
        down_clues[str(clue['num'])] = [clue['clue'] + ' ', ' ' + answer]
        grid[int(clue['cell'] / p.width)][clue['cell'] % p.width][0] = str(clue['num'])


    mydict = {'metadata': {'date': None, 'rows': p.height, 'cols': p.width}, 'clues': {'across': across_clues, 'down': down_clues}, 'grid': grid}
    return mydict

In [10]:
def find_corresponding_grid_size(data_dict, folder_name):
    for key, value in data_dict.items():
        puz_path = os.path.join(PUZ_DIR, folder_name, f"crossword_{key.replace('/', '-')}.puz")
        if os.path.exists(puz_path):
            puzzle = puz_to_json(puz_path)
            no_rows = puzzle['metadata']['rows']
            no_cols = puzzle['metadata']['cols']
            data_dict[key]['size'] = (no_rows, no_cols)
    return data_dict

In [11]:
def sort_dict_by_grid_size(data_dict):
    '''
    Sorts the given dictionary map between date and inference results according to the size 
    and creates a list of dictionary for each size possible.
    '''
    sorted_dictionary = defaultdict(list)
    for date, inf_result in data_dict.items():
        sorted_dictionary[inf_result['size']].append({'date': date, 
                                                     'result': inf_result})
    return sorted_dictionary

In [12]:
PUZ_DIR = "../Crossword Scrapper/puz"
# publication = 'Atlantic'
# if os.path.exists(PUZ_DIR):
#     puz_path_list = glob.glob(os.path.join(PUZ_DIR, publication, '*.puz'))
#     puz_path_list = [path for path in puz_path_list if '2023' in path]
    
# test_path = puz_path_list[0]
# test_path

In [16]:
folder_names = ['atlantic', 'newsday', 'the_new_yorker', 'the-LA-times']
for i, (publication, path) in enumerate(TEST_PUB_PATHS.items()):
    if i == 0:
        print('-'*100)
        
    print(publication, end = '\n\n')
    txt_path = TEST_PUB_PATHS[publication]
    all_lines = open(txt_path, 'r').readlines() # scape all the inference lines from the corresponding file

    data_dict = extract_data(all_lines) # extract all the data into a dictionary
    data_dict = find_corresponding_grid_size(data_dict, folder_names[i]) # find and add the corresponding gridsize
    try:
        sorted_dict_data = sort_dict_by_grid_size(data_dict) # sort and bucket the overall-data according to gridsize
    except:
        print(os.path.join(PUZ_DIR, folder_names[i]))

    output_data = []
    for grid_size, data_list in sorted_dict_data.items():
        total_puzzle_by_grid_size = len(data_list)
        accumulative_data = []
        for data in data_list:
            date = data['date']
            inf_data = data['result']

            if len(inf_data['Letter II']) == 0:
                f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
                f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

                s_pass_l_accu = f_pass_l_accu
                s_pass_w_accu = f_pass_w_accu
                accumulative_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
            else:
                f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
                f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

                s_pass_l_list = inf_data['Letter II']
                s_pass_w_list = inf_data['Word II']
                max_accu_index = s_pass_l_list.index(max(s_pass_l_list))

                s_pass_l_accu = round(s_pass_l_list[max_accu_index], 2)
                s_pass_w_accu = round(s_pass_w_list[max_accu_index], 2)

                if s_pass_l_accu < f_pass_l_accu:
                    s_pass_l_accu = f_pass_l_accu
                    s_pass_w_accu = f_pass_w_accu
                accumulative_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))

        df = pd.DataFrame(accumulative_data, columns = ['Date', 'First Pass Letter Accuracy', 'First Pass Word Accuracy', 'Second Pass Letter Accuracy', 'Second Pass Word Accuracy'])

        # taking a heuristic that, word accuracies lower than 85 are invalid due to crossword invalidity
        df = df[df['First Pass Word Accuracy'] > 85.0]

        # first find the number of cells which has 100% accuracies`
        pp_count_by_f_pass = len(df[df['First Pass Letter Accuracy'] == 100.0])
        pp_count_by_s_pass = len(df[df['Second Pass Letter Accuracy'] == 100.0])
        total_grids = len(df)

        avg_pp_accu_f_pass = round((pp_count_by_f_pass / total_grids) * 100, 2)
        avg_pp_accu_s_pass = round((pp_count_by_s_pass / total_grids) * 100, 2)

        f_pass_lett = round(np.mean(df['First Pass Letter Accuracy']), 2)
        f_pass_word = round(np.mean(df['First Pass Word Accuracy']), 2)

        s_pass_lett = round(np.mean(df['Second Pass Letter Accuracy']), 2)
        s_pass_word = round(np.mean(df['Second Pass Word Accuracy']), 2)

        output_data.append([publication, grid_size, total_grids, avg_pp_accu_f_pass, avg_pp_accu_s_pass, f_pass_word, s_pass_word, f_pass_lett, s_pass_lett])

    sorted_data = sorted(output_data, key=lambda x: x[1])
    pprint(sorted_data)

----------------------------------------------------------------------------------------------------
Atlantic

[['Atlantic', (5, 5), 18, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0],
 ['Atlantic', (6, 6), 20, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0],
 ['Atlantic', (7, 7), 19, 89.47, 89.47, 98.72, 98.72, 99.74, 99.74],
 ['Atlantic', (8, 8), 14, 57.14, 100.0, 95.48, 100.0, 99.0, 100.0],
 ['Atlantic', (9, 9), 9, 55.56, 88.89, 95.71, 99.18, 99.05, 99.84],
 ['Atlantic', (11, 11), 1, 0.0, 0.0, 89.47, 94.74, 97.2, 99.07],
 ['Atlantic', (12, 12), 3, 0.0, 66.67, 93.18, 97.73, 98.4, 99.44],
 ['Atlantic', (15, 15), 5, 0.0, 40.0, 92.27, 96.95, 97.9, 99.28]]
News Day

[['News Day', (15, 15), 86, 74.42, 89.53, 98.86, 99.63, 99.71, 99.92],
 ['News Day', (21, 21), 14, 78.57, 92.86, 99.23, 99.9, 99.82, 99.98]]
The LA Times

../Crossword Scrapper/puz\the_new_yorker
[['The LA Times', (15, 15), 86, 74.42, 89.53, 98.86, 99.63, 99.71, 99.92],
 ['The LA Times', (21, 21), 14, 78.57, 92.86, 99.23, 99.9, 99.82, 9

In [38]:
for path in glob.glob(os.path.join(PUZ_DIR, 'the_new_yorker', '*puz')):
    
    meta_data = puz_to_json(path)['metadata']
    if meta_data['rows'] != meta_data['cols']:
        print(meta_data)
        print(path)
        grid = puz_to_json(path)['grid']
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 'BLACK':
                    print(0, end = ' | ')
                else:
                    print(grid[i][j][1], end = ' | ')
            print('\n', '-'*60)
        
        break

{'date': None, 'rows': 15, 'cols': 16}
../Crossword Scrapper/puz\the_new_yorker\crossword_09-01-2023.puz
T | A | C | O | M | A | 0 | W | A | R | T | 0 | P | U | L | L | 
 ------------------------------------------------------------
I | S | O | B | A | R | 0 | I | N | E | E | D | A | N | A | P | 
 ------------------------------------------------------------
S | H | R | I | N | K | I | N | G | F | E | E | L | I | N | G | 
 ------------------------------------------------------------
H | E | N | S | 0 | S | O | O | 0 | R | N | A | 0 | T | E | A | 
 ------------------------------------------------------------
0 | 0 | B | P | A | 0 | T | U | B | A | 0 | L | E | S | S | 0 | 
 ------------------------------------------------------------
C | H | R | O | M | E | A | T | A | C | O | S | T | 0 | 0 | 0 | 
 ------------------------------------------------------------
R | O | E | 0 | M | R | S | 0 | A | T | M | 0 | T | C | B | Y | 
 ------------------------------------------------------------
O | R 